# 판매자 전처리

In [ ]:
import pandas as pd
import os
from pathlib import Path
from src.database import load_table

OUTPUT_DIR = Path("../data/processed")

In [ ]:
# 1. 데이터 불러오기
try:
    # Supabase에서 판매자 데이터 직접 로드
    merged = load_table('sellers')
    
    print(f"✅ 데이터 로드 완료: sellers({len(merged)}건)")
except Exception as e:
    print(f"❌ 데이터를 로드하는 중 오류가 발생했습니다: {e}")


In [ ]:
# 2. 주소 데이터 추출 (business_address에서 '시 구' 추출)
def extract_city_district(address):
    if not address or not isinstance(address, str):
        return None
    # 첫 두 단어(시/도, 구/군) 추출
    parts = address.split()
    if len(parts) >= 2:
        return f"{parts[0]} {parts[1]}"
    return parts[0] if parts else None

merged['city_state'] = merged['business_address'].apply(extract_city_district)
print("✅ business_address에서 city_state 추출 완료")

In [ ]:
# 3. 불필요한 컬럼 삭제
cols_to_drop = [
    'business_address',              # 정제된 주소(city_state)로 대체
    'business_registration_number', 
    'email', 
    'phone', 
    'mail_order_number', 
    'created_at'
]
# 실제 데이터에 존재하는 컬럼만 골라 삭제합니다.
merged = merged.drop(columns=[c for c in cols_to_drop if c in merged.columns])


In [ ]:
# 4. 판매자 이름 정제 (is_english_name 생성)
if 'representative_name' in merged.columns:
    print("✓ 'representative_name' 컬럼을 기준으로 이름 정제를 수행합니다.")
    # 영문(a-z, A-Z)이 포함되어 있으면 1, 아니면 0
    merged['is_english_name'] = merged['representative_name'].str.contains('[a-zA-Z]', regex=True).fillna(False).astype(int)
else:
    print("⚠️ 'representative_name' 컬럼을 찾을 수 없습니다.")
    merged['is_english_name'] = 0

In [ ]:
# 5. 컬럼 순서 정리 (보기 좋게 배치)
important_cols = ['id', 'company_name', 'representative_name', 'is_english_name', 'city_state']
existing_important = [c for c in important_cols if c in merged.columns]
other_cols = [c for c in merged.columns if c not in existing_important]
merged = merged[existing_important + other_cols]

In [ ]:
# 6. 최종 파일 저장
output_path = OUTPUT_DIR / 'sellers_final_cleaned.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
merged.to_csv(output_path, index=False, encoding='utf-8-sig')

print("\n✨ 전처리 파이프라인 완료!")
print(f"✓ 최종 저장 경로: {output_path}")
print(f"✓ 총 데이터 개수: {len(merged)} 행")

## 모델링용 추가 전처리

In [ ]:
 # 3. 해외 사업자 여부 생성
korea_keywords = ['서울', '경기', '인천', '부산', '대구', '광주', '대전', '울산', '세종', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주']
merged['is_cross_border'] = merged['city_state'].apply(
    lambda x: 0 if any(k in str(x) for k in korea_keywords) else 1
)

In [ ]:
# 4. 판매자 이름 정제 (이름에 영문 포함 + 해외 주소인 경우만 1)
if 'representative_name' in merged.columns:
    merged['is_english_name'] = merged.apply(
        lambda x: 1 if (pd.Series(str(x['representative_name'])).str.contains('[a-zA-Z]', regex=True).iloc[0] 
                       and x['is_cross_border'] == 1) else 0, axis=1
    )


In [ ]:
# 5. [중요] 주소 고유 종류(nunique) 확인
unique_address_count = merged['city_state'].nunique()

print("\n" + "="*50)
print("📊 주소 종류 분석 결과")
print(f"👉 중복을 제외한 주소(구/도시) 종류: {unique_address_count}개")
print("="*50 + "\n")

In [ ]:
# 6. 불필요한 텍스트 컬럼 삭제 (모델링용 전단계)
cols_to_drop = ['representative_name', 'business_address', 'company_name', 
                'business_registration_number', 'email', 'phone', 
                'mail_order_number', 'created_at']
existing_drops = [c for c in cols_to_drop if c in merged.columns]
ml_ready_df = merged.drop(columns=existing_drops)

In [ ]:
# 7. 최종 저장
output_path = OUTPUT_DIR / 'sellers_ml_ready.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
ml_ready_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print("✅ 처리가 완료되었습니다!")
print(f"✅ 저장 위치: {output_path}")